In [2]:
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

##### 01. 데이터프레임 생성
- 영수증 전처리

In [2]:
receipt_data = '''
이마트 청계천절(0212290-1234
06030 【  208-80-60013 이갑수
서울시 종구 청계천로 400
영수증 미지참시 교환/환불 불가(30일내)
환불 구매점에서 가능(결제키느시잠)
체크키드 취소 시 대금환급은 최대 7일소요

2014206004 11113       60005

미 (0)치킨 너켓 19      10,500
02 용가리치킨520+140      8.550
03 대림 안심지킨너540  7,980
04 치리 기슴살치킨너켓  5,990
수실류-호두
호두
너겟
가슬살
치킨
자몽

7.80

부기세 면세 품목입니다.

0088.000
'''
receipt = receipt_data.split('\n')
receipt_df = pd.DataFrame(receipt, columns=['title'])
receipt_df['title'] = receipt_df['title'].apply(lambda x: re.sub('[^가-힣]', '', x))
receipt_df = receipt_df[receipt_df.title != '']
receipt_df.set_index('title', inplace=True)
receipt_df.reset_index(inplace=True)
receipt_df

,title
0,이마트청계천절
1,이갑수
2,서울시종구청계천로
3,영수증미지참시교환환불불가일내
4,환불구매점에서가능결제키느시잠
5,체크키드취소시대금환급은최대일소요
6,미치킨너켓
7,용가리치킨
8,대림안심지킨너
9,치리기슴살치킨너켓


- 리스트 전처리

In [3]:
# 전처리 전 데이터 불러오기
list_df = pd.read_csv('data/친환경 제품(초안).csv')
# 중복 데이터 제거
list_df.drop_duplicates(subset=['title'], inplace=True)
# 불필요한 문자 제거
list_df['title_mod'] = list_df['title'].apply(lambda x: re.sub(r'\([^)]*\)', '', x))
list_df['title_mod'] = list_df['title_mod'].apply(lambda x: re.sub(r'\[[^)]*\]', '', x))
list_df['title_mod'] = list_df['title_mod'].apply(lambda x: re.sub(r'[^ ㄱ-ㅣ가-힣]+', '', x))
list_df['point'] = list_df['point'].apply(lambda x: re.sub(r'[^0-9]+', '', x))
list_df['point'] = list_df['point'].astype(int)
# 인덱스 정렬
list_df.set_index('title', inplace=True)
list_df.reset_index(inplace=True)
# 컬럼별 공백 제거
list_df['market'] = list_df['market'].str.lstrip(' ')
list_df['title'] = list_df['title'].str.lstrip(' ')
list_df['title_mod'] = list_df['title_mod'].str.lstrip(' ')
# 최종본으로 저장
list_df.to_csv('data/eco_product.csv', index=False)
list_df.head()

,title,point,market,title_mod
0,수실류-은행,10,로컬푸드 직매장,수실류은행
1,수실류-호두,10,로컬푸드 직매장,수실류호두
2,수실류-대추,10,로컬푸드 직매장,수실류대추
3,수실류-밤,10,로컬푸드 직매장,수실류밤
4,과실류-기타,10,로컬푸드 직매장,과실류기타


##### 02. 유사도 검사
- TF-IDF 벡터화

In [7]:
tv = TfidfVectorizer()
receipt_tv = tv.fit_transform(receipt_df['title'])
list_tv = tv.transform(list_df['title_mod'])

- 코사인 유사도 계산

In [8]:
cosine_similarities = cosine_similarity(receipt_tv, list_tv)


- 유사도 컬럼으로 추가

In [9]:
receipt_df['cosine_similarity'] = cosine_similarities.max(axis=1)

- 유사한 항목 추가

In [10]:
most_similar_indices = cosine_similarities.argmax(axis=1)
most_similar_items = list_df.loc[most_similar_indices, 'title_mod'].values
receipt_df['most_similar_item'] = most_similar_items
receipt_df['point'] = list_df.loc[most_similar_indices, 'point'].values
receipt_df['market'] = list_df.loc[most_similar_indices, 'market'].values

In [11]:
receipt_df

,title,cosine_similarity,most_similar_item,point,market
0,이마트청계천절,0.0,수실류은행,10,로컬푸드 직매장
1,이갑수,0.0,수실류은행,10,로컬푸드 직매장
2,서울시종구청계천로,0.0,수실류은행,10,로컬푸드 직매장
3,영수증미지참시교환환불불가일내,0.0,수실류은행,10,로컬푸드 직매장
4,환불구매점에서가능결제키느시잠,0.0,수실류은행,10,로컬푸드 직매장
5,체크키드취소시대금환급은최대일소요,0.0,수실류은행,10,로컬푸드 직매장
6,미치킨너켓,0.0,수실류은행,10,로컬푸드 직매장
7,용가리치킨,0.0,수실류은행,10,로컬푸드 직매장
8,대림안심지킨너,0.0,수실류은행,10,로컬푸드 직매장
9,치리기슴살치킨너켓,0.0,수실류은행,10,로컬푸드 직매장


In [12]:
# cosine_similarity의 값이 0.5 이상인 경우만 추출
receipt_df = receipt_df[receipt_df['cosine_similarity'] >= 0.5]
receipt_df

,title,cosine_similarity,most_similar_item,point,market
10,수실류호두,1.0,수실류호두,10,로컬푸드 직매장
11,호두,1.0,정월대보름 껍질땅콩 호두,670,올가홀푸드
15,자몽,1.0,프레스드 자몽,440,흥국에프엔비


In [13]:
# 포인트 합계
receipt_df['point'].sum()

1120

def get_similarty(list, receipt)